#### Author : Gautam Badri

# 3) Dynamic Convolutional Neural Network

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### 3A) Create a NN with dynamic configuration


### 3B) config parameter provides list of tuples, each tuple specifics Conv2d() configuration


### 3C) Determine size of FC layer, programatically (print size of tensor after flowing through above convolutions and multiply those sizes)


### 3D) Do softmax before returning in the forward function.

In [41]:
class Dyn_CNN(nn.Module):
    def __init__(self, config, input_dim, num_classes):
        super().__init__()
        self.input_dim = input_dim
        self.num_layers = len(config)
        self.conv_layers = nn.ModuleList(
            [nn.Conv2d(in_channels=i, out_channels=o, stride=s, kernel_size=k, padding=p) for i, o, s, k, p in config])
        self.relu_layers = nn.ModuleList([nn.ReLU() for i in range(len(config))])
        self.fc_in_h = input_dim[0]
        self.fc_in_w = input_dim[1]
        for i,o,s,f,p in config:       #stride=(heightxwidth)
            self.fc_in_h = ((self.fc_in_h-f[0]+2*p)//s)+1
            self.fc_in_w = ((self.fc_in_w-f[1]+2*p)//s)+1
        # print('in_features = ' , config[-1][1]*self.fc_in_h*self.fc_in_w)
        self.fc = nn.Linear(in_features = config[-1][1]*self.fc_in_h*self.fc_in_w, out_features=num_classes)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        self.conv_out_feature_maps = 0
        for i,l in enumerate(self.conv_layers):
            x = self.conv_layers[i](x)
            x = self.relu_layers[i](x)
        x = self.flatten(x)
        # print(x.shape)
        x = self.fc(x)
        x = self.softmax(x)
        return x

In [4]:
def get_datasets():
    train_data = datasets.FashionMNIST(
        root="data",
        train = True,
        download = True,
        transform = ToTensor()
    )

    test_data = datasets.FashionMNIST(
        root = "data",
        train = True,
        download = True,
        transform = ToTensor()
    )
    
    return train_data, test_data

In [5]:
def get_dataloaders(train_data, test_data):
    train_dataloader = DataLoader(train_data, batch_size=64)
    test_dataloader = DataLoader(test_data, batch_size=64)
    return train_dataloader, test_dataloader

### 3E) Customized loss function (not the library version)

In [6]:
def loss_fun(y_pred, y_actual):
  v = -(y_actual * torch.log(y_pred+1e-10))
  v = torch.sum(v)
  return v

In [7]:
def get_optim(model, lr=1e-6):
    optim = torch.optim.SGD(model.parameters(), lr = lr)
    return optim

In [8]:
def get_num_classes(train_data):
    return len(train_data.classes)

In [9]:
def get_input_dim(train_dataloader):
    _x, _y = None,None

    for X, y in test_dataloader:
        _x = X.shape
        _y = y.shape
        print(f"Shape of X: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
        break
    
    return _x[2],_x[3] #heightxwidth

In [10]:
def get_model(config, input_dim, num_classes):
    model = Dyn_CNN(config, input_dim, num_classes)
    return model

In [25]:
def train_network(train_dataloader, model, optim, loss_fn, epochs=5):
    print('Training Model ...\n\n')
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            inputs , labels = inputs.to(device), labels.to(device)
            optim.zero_grad()
            outputs = model(inputs)
            tmp = torch.nn.functional.one_hot(labels, num_classes= 10)
            loss = loss_fn(outputs, tmp)
            loss.backward()
            optim.step()
            running_loss += loss.item()
            # return
        print("[Epoch : {}/{}] loss = ".format(epoch+1,epochs),running_loss)

In [12]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [13]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from torchmetrics import Precision, Recall, F1Score, Accuracy
from torchmetrics.classification import accuracy

In [15]:
def test_network(dataloader, model, loss_fun):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            tmp = torch.nn.functional.one_hot(y, num_classes= 10)
            pred = model(X)
            test_loss += loss_fun(pred, tmp).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/= num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    accuracy1 = Accuracy().to(device)
    print('Accuracy :', accuracy1(pred,y))
    precision = Precision(average = 'macro', num_classes = 10).to(device)
    print('precision :', precision(pred,y))

    recall = Recall(average = 'macro', num_classes = 10).to(device)
    print('recall :', recall(pred,y))
    f1_score = F1Score(average = 'macro', num_classes = 10).to(device)
    print('f1_score :', f1_score(pred,y))
    return accuracy1,precision, recall, f1_score

In [16]:
train_data, test_data = get_datasets()
train_dataloader, test_dataloader = get_dataloaders(train_data, test_data)

In [17]:
num_classes = get_num_classes(test_data)

In [18]:
num_classes

10

In [19]:
input_dim = get_input_dim(train_dataloader)

Shape of X: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [20]:
input_dim

(28, 28)

In [21]:
#in_channels=i, out_channels=o, stride=s, kernel_size=k, padding=p
config = [(1, 20, 1, (5,5), 0), (20, 50, 1, (5,5), 0)]

In [42]:
# config, input_dim, num_classes
model = get_model(config, input_dim, num_classes)
model = model.to(device)

### 3F) Train on training data set

In [43]:
optim = get_optim(model)
train_network(train_dataloader, model, optim, loss_fun)

Training Model ...


[Epoch : 1/5] loss =  133944.6523590088
[Epoch : 2/5] loss =  123956.33857727051
[Epoch : 3/5] loss =  106795.77738952637
[Epoch : 4/5] loss =  85399.45009613037
[Epoch : 5/5] loss =  69562.80276870728


### 3G) Test on the test data set - report accuracy, precision, recall and F1 scores.

In [44]:
test_network(test_dataloader, model, loss_fun)

Test Error: 
 Accuracy: 67.3%, Avg loss: 68.419176 

Accuracy : tensor(0.7188, device='cuda:0')
precision : tensor(0.6817, device='cuda:0')
recall : tensor(0.7583, device='cuda:0')
f1_score : tensor(0.7086, device='cuda:0')


(Accuracy(), Precision(), Recall(), F1Score())